In [17]:
#Bibliotecas já instaladas no ambiente Python
import os
import time
#Importar as funções que iremos utilizar do Selenium
from selenium import webdriver
#Importando BeautifulSoup
from bs4 import BeautifulSoup

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [19]:
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install())

#Atribuir ao Selenium o site que será acessado
browser.get('https://tempo.inmet.gov.br/TabelaEstacoes/')

#Recomendado para evitar ban do servidor
time.sleep(3)

#Selecionar em "Produto" a opção "Tabela de Dados das Estações" 
browser.find_element_by_xpath("//select/option[@value='TabelaEstacoes']").click()


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\lucap\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [20]:
VALOR_ESTACAO='A001' #Porto Alegre - RS

DATA_INIT = '11/10/2021'
DATA_END = '11/10/2021'

#Selecionamos a opção de estação automática
browser.find_element_by_xpath("//select/option[@value='T']").click()

#Selecionamos qual estação estamos interessados
browser.find_element_by_xpath("//select/option[@value='"+VALOR_ESTACAO+"']").click()

#Primeiro limpados o formulário e então preenchemos com a data inicial que desejamos.

browser.find_element_by_id("datepicker_EstacoesTabela_Inicio").clear()
browser.find_element_by_id("datepicker_EstacoesTabela_Inicio").send_keys(DATA_INIT)

#O mesmo para a data final
browser.find_element_by_id("datepicker_EstacoesTabela_Fim").clear()
browser.find_element_by_id("datepicker_EstacoesTabela_Fim").send_keys(DATA_END)

#Por fim, clicamos em "gerar tabela"
browser.find_element_by_id("EstacoesTabela").click()

In [21]:
#Pedimos para o Selenium aguardar por alguns segundos até que a tabela seja gerada pelo site
WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.ID, "tabela")))

#atribuimos a estrutura atual do site para uma variável para que o BeautifulSoup possa fazer sua mágica!
page_source = browser.page_source

#"Limpamos" a estrutura do site com o BeautifulSoup 
soup = BeautifulSoup(page_source, 'lxml')

#Pede para retornar a tabela existente na tag "table"
table = soup.find('table')

In [22]:
import pandas as pd

#Converter a tabela html em Dataframe. 
#Definimos como separador decimal ',' e milhar '.'
df = pd.read_html(str(table),decimal=',', thousands='.')[0]
df=pd.DataFrame(df.to_records())

#Se preferir, também há como renomear o cabeçalho da planilha.

New_Names=['index','Date','Time','Temp_inst','Temp_max','Temp_min','Umi_Rinst','Umi_Rmax','Umi_Rmin','PtOrvalho_inst','PtOrvalho_max','PtOrvalh_min',
          'Pressão_inst','Pressão_max','Pressão_min','Vento_veloc','Vento_dir','Vento_raj','Rad','Precipt'
          ]

for n in  range(0,len(df.keys())):
    df = df.rename(columns={df.keys()[n]: New_Names[n]})
    

In [23]:
df['Date'] = df['Date'] +' '+ df['Time'].apply(lambda x: '{}:00:00'.format(int(x)/100))
df['Date'] = pd.to_datetime(df['Date'])
df.drop(['Time','index'],axis=1,inplace=True)
df.set_index(['Date'],inplace=True)

In [24]:
df

,Temp_inst,Temp_max,Temp_min,Umi_Rinst,Umi_Rmax,Umi_Rmin,PtOrvalho_inst,PtOrvalho_max,PtOrvalh_min,Pressão_inst,Pressão_max,Pressão_min,Vento_veloc,Vento_dir,Vento_raj,Rad,Precipt
Date,,,,,,,,,,,,,,,,,
2021-11-10 00:00:00,18.7,18.8,18.4,92.0,93.0,91.0,17.5,17.6,17.1,886.9,887.1,886.8,2.2,179.0,4.9,NaN,2.2
2021-11-10 01:00:00,18.6,18.8,18.5,92.0,93.0,91.0,17.2,17.5,17.1,887.6,887.6,886.8,2.2,30.0,4.9,NaN,0.0
2021-11-10 02:00:00,18.6,18.7,18.3,93.0,93.0,91.0,17.4,17.5,17.1,888.1,888.3,887.6,1.9,283.0,4.1,NaN,0.0
2021-11-10 03:00:00,18.4,18.6,18.2,92.0,93.0,90.0,17.1,17.3,16.8,887.7,888.1,887.7,2.2,299.0,4.6,NaN,0.2
2021-11-10 04:00:00,18.8,18.8,18.3,93.0,93.0,92.0,17.6,17.6,17.0,887.5,887.7,887.5,1.4,272.0,4.6,NaN,0.2
2021-11-10 05:00:00,18.9,19.0,18.7,93.0,93.0,93.0,17.8,17.8,17.6,887.7,887.8,887.5,1.3,182.0,2.9,NaN,0.6
2021-11-10 06:00:00,18.7,18.9,18.7,94.0,94.0,93.0,17.7,17.8,17.7,886.8,887.8,886.7,1.1,98.0,2.9,NaN,0.8
2021-11-10 07:00:00,18.4,18.8,18.4,94.0,94.0,94.0,17.5,17.8,17.5,885.6,886.8,885.4,3.2,351.0,5.4,NaN,0.0
2021-11-10 08:00:00,18.4,18.4,18.3,92.0,94.0,92.0,17.1,17.5,17.1,886.3,886.3,885.6,1.0,36.0,5.4,NaN,0.2
